In [3]:
from threading import Thread
import os
from xmlrpc.server import SimpleXMLRPCServer

import numpy as np
mport specpy as sp

import autosted
from autosted.callback_buildingblocks.coordinate_value_wrappers import StageOffsetsSettingsGenerator
from autosted.callback_buildingblocks.parameter_filtering import LocationRemover
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader
from autosted.callback_buildingblocks.regular_position_generators import PositionListOffsetGenerator
from autosted.taskgeneration.timeseries import TimeSeriesCallback
from autosted.utils.tiling import centered_tiles

class ImspectorAcquisition():

    def __init__(self, save_dir, file_prefix) -> None:
        self.imspector = sp.get_application()

        # TODO: catch no open measurement
        self.parameters = self.imspector.active_measurement().active_configuration().parameters('')

        self.position = [self.imspector.active_measurement().active_configuration().parameters('ExpControl/scan/range/coarse_' + c + '/g_off') for c in 'zyx']
        self.field_of_view = [self.imspector.active_measurement().active_configuration().parameters('ExpControl/scan/range/' + c + '/len') for c in 'zyx']
        self.overlap = 0.1

        self.save_path = save_dir
        self.file_prefix = file_prefix

        self.n_tiles = 1
        self.time_points = None

        self.pipeline_thread = None

    def start_pipeline_thread(self):

        if self.pipeline_thread is not None:
            raise ValueError('a measurement pipeline is running currently, finish that first.')

        self.pipeline_thread = Thread(target=self.run_pipeline)
        self.pipeline_thread.start()

    @staticmethod
    def get_smallest_numeric_suffix_for_files(base_h5_path):
        suffix = 0
        while os.path.exists(base_h5_path.replace('.h5', f'{suffix}.h5')):
            suffix += 1
        return suffix

    def run_pipeline(self):

        # make output dir if it does not exist already
        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)

        levels = ()
        # add timepoint level if we want timepoints
        if self.time_points is not None:
            levels += ('timepoint', )
        # call second level tile if we have tiles, else image
        levels += (('tile', ) if np.max(self.n_tiles) > 1 else ('image', ))

        # init pipeline
        pl = autosted.AcquisitionPipeline(self.save_path, hierarchy_levels=levels)

        # make name handler, add suffix to prevent overwriting exisiting files
        numeric_suffix = ImspectorAcquisition.get_smallest_numeric_suffix_for_files(os.path.join(self.save_path, self.file_prefix + '.h5'))
        pl.filename_handler = autosted.FilenameHandler(self.save_path, levels, prefix=f'{self.file_prefix}{numeric_suffix}')


        tile_positions = centered_tiles(self.position, self.field_of_view, (1, self.n_tiles, self.n_tiles), self.overlap)
        tile_generator = PositionListOffsetGenerator(tile_positions)
        atg_images = autosted.AcquisitionTaskGenerator(levels[-1],
            LocationRemover(JSONSettingsLoader([self.parameters])),
            StageOffsetsSettingsGenerator(tile_generator.get_all_locations, True)
        )

        if self.time_points is not None:
            timeseries_callback = TimeSeriesCallback("timepoint")
            timeseries_callback.time_points = self.time_points
            pl.add_callback(timeseries_callback, "timepoint")

            # also add tile callback
            pl.add_callback(atg_images, "timepoint")

        if self.time_points is not None:
            pl.run(timeseries_callback)
        else:
            pl.run(atg_images)

    def set_parameters_and_start_acquisition(self, n_tiles, time_points):

        self.n_tiles = n_tiles
        self.time_points = time_points

        self.start_pipeline_thread()

    def finish_acquisition(self):
        if self.pipeline_thread is None:
            return
        self.pipeline_thread.join()
        self.pipeline_thread = None

    def run_server(self, host, port):
        with SimpleXMLRPCServer((host, port), allow_none=True) as server:
            server.register_function(self.finish_acquisition, 'finish_acquisition')
            server.register_function(self.set_parameters_and_start_acquisition, 'run_acquisition')
            server.serve_forever()



## Run

In [ ]:
save_path = 'D:/AUTOMATION/microfluidic/20230830_hyb+strip_seq1-seq3/'
file_prefix = 'run'
acq = ImspectorAcquisition(save_path, file_prefix)

acq.run_server('10.163.69.197', 11005)

## Tests

Using the cells below, you can run an acquisition locally in blocking or threaded mode (or via XMLRPC as above)

In [ ]:
# setup acquisition

save_path = 'D:/AUTOMATION/microfluidic/20230403_fish_test_3/'
file_prefix = 'dead_time_test'
acq = ImspectorAcquisition(save_path, file_prefix)

### Run blocking

In [ ]:
acq.time_points = [0, 10, 20]
acq.n_tiles = (1, 2)
acq.overlap = 0.25
acq.run_pipeline()

### Run in seperate thread

In [ ]:
acq.time_points = [0, 10, 20]
acq.n_tiles = (1, 2)
acq.start_pipeline_thread()

In [ ]:
# blocking merge with the acquisition thread 
acq.finish_acquisition()

### Run as XMLRPC server

In [ ]:
import netifaces

netifaces.interfaces()

In [ ]:
acq.run_server('10.163.69.197', 11005)